## Import

In [294]:
import numpy as np
import pandas as pd
from sklearn import datasets 

## Data (Iris)

In [295]:
if 1:
    iris = datasets.load_iris()
    data = pd.DataFrame(iris.data)
    data.columns = iris.feature_names
    target = pd.DataFrame(iris.target)
elif 1:
    iris = datasets.load_linnerud()
    data = pd.DataFrame(iris.data)
    data.columns = iris.feature_names
    target = pd.DataFrame(iris.target)

print(data.head())
print(target.head())

   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0                5.1               3.5                1.4               0.2
1                4.9               3.0                1.4               0.2
2                4.7               3.2                1.3               0.2
3                4.6               3.1                1.5               0.2
4                5.0               3.6                1.4               0.2
   0
0  0
1  0
2  0
3  0
4  0


## Nodes 
Each node of the tree, ending in a leaf node. 

In [296]:
class Branch():
    def __init__(self, feature=None, thresh=None, left=None, right=None, info_gain=None, value=None):
        # non leaf node (decision node)
        self.feature = feature
        self.thresh = thresh
        self.left = left
        self.right = right
        self.info_gain = info_gain
        # leaf node
        self.value = value

## Trees 
For this NB ill only be using a single tree. 

In [297]:
class Tree():
    def __init__(self, min_samples_split=2, max_depth=2):        
        self.root = None # keep track of root node
        # parameters
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth=0):
        # recusivly build tree
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)

        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            #print(best_split)
            # check if information gain is positive
            if best_split != {}:
                if best_split["info_gain"]>0:
                    # recur left
                    left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                    # recur right
                    right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                    # return decision node
                    return Branch(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["info_gain"])
    
        leaf_value = self.calculate_leaf_value(Y)
        return Branch(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        best_split = {}
        max_info_gain = -float(999999999)
        # features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            # feature values present in the data
            for threshold in possible_thresholds:
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if childs are not empty
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # information gain
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    # update the best split if needed
                    if curr_info_gain > max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
                        
        # return best split
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def information_gain(self, parent, l_child, r_child, mode="entropy"):
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode=="gini":
            gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        return gain
    
    def entropy(self, y):
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy
    
    def gini_index(self, y):
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini
        
    def calculate_leaf_value(self, Y):
        Y = list(Y)
        return max(Y, key=Y.count)
    
    def print_tree(self, tree=None, indent=" "):
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature), "<=", tree.thresh, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def fit(self, X, Y):
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
    
    def predict(self, X):
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions
    
    def make_prediction(self, x, tree):
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature]
        if feature_val<=tree.thresh:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

## Split train/test data

In [298]:
X = data.values
Y = target
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=111)

## Fit Model 

In [299]:
model = Tree(max_depth=4, min_samples_split=2)
model.fit(X_train, Y_train)
model.print_tree()

X_2 <= 1.9 ? 0.33395833333333336
 left:0.0
 right:X_3 <= 1.6 ? 0.42707880434782614
  left:X_2 <= 4.9 ? 0.08931947069943283
    left:1.0
    right:X_0 <= 6.0 ? 0.375
        left:1.0
        right:2.0
  right:2.0


## Get accuracy with sklearn

In [300]:
Y_pred = model.predict(X_test) 
print(Y_pred)
print(Y_test)
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_pred)

[0.0, 0.0, 2.0, 2.0, 2.0, 0.0, 0.0, 2.0, 2.0, 1.0, 2.0, 0.0, 1.0, 2.0, 2.0, 0.0, 2.0, 1.0, 0.0, 2.0, 1.0, 2.0, 1.0, 1.0, 2.0, 0.0, 0.0, 2.0, 0.0, 2.0]
     0
39   0
26   0
109  2
123  2
77   1
24   0
0    0
139  2
144  2
56   1
131  2
36   0
119  2
124  2
105  2
15   0
126  2
57   1
35   0
140  2
73   1
137  2
55   1
82   1
113  2
23   0
25   0
70   1
2    0
136  2


0.9

## Random Forest

In [301]:
from scipy import stats

class Forest():
    def __init__(self, num_trees = 5, min_samples_split=2, max_depth=2):
        ''' constructor '''
        
        # initialize the number of trees
        self.num_trees = num_trees
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth

    def fit(self, X_train, Y_train):
        ''' function to train the entire forest '''
        
        # list to store the individual decision trees
        forest = []
        for i in range(self.num_trees):
            tree = Tree(min_samples_split=self.min_samples_split, max_depth=self.max_depth)
            X_train_new, Y_train_new = self.bootstrap_sample(X_train, Y_train)
            #X_train_new = self.drop_features(X_train_new)
            tree.fit(X_train_new, Y_train_new)
            forest.append(tree)
            #print(i)
        
        self.forest = forest

    def bootstrap_sample(self, x, y):
        x = np.array(x)
        y = np.array(y)
        shrink = 0.9
        num_samples, num_features = np.shape(x)
        x_train_new, y_train_new = np.zeros((int(num_samples*shrink), num_features)), np.zeros((int(num_samples*shrink), 1))

        for i in range(int(num_samples*shrink)):
            idx = np.random.randint(0, num_samples)
            x_train_new[i, :] = x[idx, :]
            y_train_new[i, :] = y[idx, :]
            
        return x_train_new, y_train_new


    def drop_features(self, x_train, drop_percent=0.3):
        num_samples, num_features = np.shape(x_train)
        num_drop = int(num_features*drop_percent)
        if num_drop != 0:
            drop_indices = np.random.choice(np.arange(num_features), num_drop, replace=False)
            x_train = np.delete(x_train, drop_indices, axis=1)
        return x_train
    

    def predict_trees(self, X_test):
        predictions = []
        for tree in self.forest:
            pred = tree.predict(X_test)
            predictions.append(pred)
        return predictions


    def predict(self, X_test):
        pred = self.predict_trees(X_test)
        #print("predictions: ", pred)
        predictions = stats.mode(pred, keepdims=True)[0][0]
        return predictions


    def print_forest(self):

        for i, tree in enumerate(self.forest) :
            #print("\nTree: ", i, "\n")
            tree.print_tree()



In [309]:
for x in range(0, 15):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=(x*13+3))
    forest = Forest(num_trees=(x*10 + 1), min_samples_split=3, max_depth=3)
    forest.fit(X_train, Y_train)
    #forest.print_forest()
    predictions = forest.predict(X_test)

    #print("final predictions: ", predictions)
    #print("Y_train: ", Y_test)
    print("Accuracy score (", x*20, " trees): ", 100 * accuracy_score(Y_test, predictions), "%")

Accuracy score ( 0  trees):  93.33333333333333 %
Accuracy score ( 20  trees):  96.66666666666667 %
Accuracy score ( 40  trees):  90.0 %
Accuracy score ( 60  trees):  100.0 %
Accuracy score ( 80  trees):  96.66666666666667 %
Accuracy score ( 100  trees):  96.66666666666667 %
Accuracy score ( 120  trees):  100.0 %
Accuracy score ( 140  trees):  90.0 %
Accuracy score ( 160  trees):  100.0 %
Accuracy score ( 180  trees):  100.0 %
Accuracy score ( 200  trees):  96.66666666666667 %
Accuracy score ( 220  trees):  96.66666666666667 %
Accuracy score ( 240  trees):  93.33333333333333 %
Accuracy score ( 260  trees):  86.66666666666667 %
Accuracy score ( 280  trees):  93.33333333333333 %
